In [1]:
import import_ipynb
from Pushkin_v2 import *

model_up = CharRNNLoop()
model_up.load_state_dict(torch.load('text_generation_pushkin_RNN.h5'))
model_up.eval()

importing Jupyter notebook from Pushkin_v2.ipynb


CharRNNLoop(
  (emb): Embedding(8343, 16)
  (rnn): RNN(16, 64, batch_first=True)
  (hid_to_logits): Linear(in_features=64, out_features=8343, bias=True)
)

In [2]:
import time
import sys
import requests
import random
from bs4 import BeautifulSoup
import telebot
from requests.packages.urllib3.exceptions import InsecureRequestWarning

import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

telegram_token = '2081251162:AAGXKcvJyS5jpDM6vFD_KDF-iAv0Kf1Lb1A'

log_path = 'poem_log.txt'

In [ ]:
# def init_html_session():
#     """Инициализируем сессию, чтобы не логиниться при каждом запросе заново"""
#     s = requests.sessions.Session()
#     s.verify = False
#     return s


# def get_json_from_url(url, session):
#     """get json from url"""
#     response = session.get(url)
#     response.raise_for_status()
#     return response.content


# def write_log(num, code, name):
#     with open(log_path, encoding='utf-8', mode='a+') as logfile:
#         logfile.write(f'\n{num} || {code} || {name}')


# def read_log():
#     with open(log_path, encoding='utf-8', mode='r+') as logfile:
#         return [it.split('||')[0].strip() for it in logfile.read().split('\n')]


# def send_message(chat_id, text):
#     method = "sendMessage"
#     url = f"https://api.telegram.org/bot{telegram_token}/{method}"
#     data = {"chat_id": chat_id, "text": text, "parse_mode": 'html'}
#     requests.post(url, data=data)


# def get_unused_poem():
#     used_id = read_log()
#     random_id = '1'
#     while random_id in used_id:
#         random_id = str(random.randint(1, 100000))

#     return random_id


# def get_poem():
#     session = init_html_session()

#     status_code = 0
#     random_id = 0
#     poem_name = ''
#     response = requests.Response()
#     while status_code != 200:
#         random_id = get_unused_poem()
#         response = session.get(f'https://www.culture.ru/poems/{random_id}')
#         status_code = response.status_code
#         if status_code == 200:
#             poem_name = BeautifulSoup(response.content, 'html.parser').find('h1',
#                                                                             {'class': "entity-heading_title"}).text
#         write_log(random_id, status_code, poem_name)
#         time.sleep(0.2)

#     return response.content


# https://stackoverflow.com/questions/57568979/getting-input-from-users-in-bot
# https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/echobot.py
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Привет {user.mention_markdown_v2()}\! Напиши мне начало фразы, и я постараюсь продолжить в стиле Пушкина',
        reply_markup=ForceReply(selective=True),
    )
    
def do_something(user_input):
    answer, words = generate_text(model_up, tokens, seed_phrase = 'sosos ' + user_input.lower(), max_length=150, temperature=1.0)
    words = 'Я не знаю слово "' + ' '.join([token if token not in tokens else '' for token in words]) + '"'
    if (words.replace(' ', '') == 'Янезнаюслово""'):
        words = 'Моя реприза:'
    return answer, words

def reply(update, context):
    user_input = update.message.text
    update.message.reply_text(do_something(user_input)[1])
    update.message.reply_text(do_something(user_input)[0])
    
# https://stackoverflow.com/questions/45537938/pythonbot-pass-argument-to-command-hanlder
def generate_seq(update: Update, context: CallbackContext):
    while True:
        try:
            N = int(context.args[0])
        except ValueError:
            update.message.reply_text('Параметр должен быть ЧИСЛОМ')
            break
        else:
            if N > 0:
                update.message.reply_text('Последовательность моих реприз:')
                for i in range(N):
                    update.message.reply_text(do_something('')[0])
            else:
                update.message.reply_text('Параметр должен быть ПОЛОЖИТЕЛЬНЫМ целым числом')
            break
            
def generate(update: Update, context: CallbackContext):
    while True:
        try:
            N = int(context.args[0])
        except ValueError:
            update.message.reply_text('Параметр должен быть ЧИСЛОМ')
            break
        else:
            if N > 0:
                update.message.reply_text('Последовательность моих реприз:')
                cnt = 0
                while cnt < N:
                    text = do_something('')[0]
                    update.message.reply_text(text)
                    cnt += len(text)
            else:
                update.message.reply_text('Параметр должен быть ПОЛОЖИТЕЛЬНЫМ целым числом')
            break
            

def main():
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater(telegram_token, use_context=True)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher
    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
#     dispatcher.add_handler(MessageHandler(Filters.text, reply))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, reply))
    
    dispatcher.add_handler(CommandHandler("generate_seq", generate_seq))
    dispatcher.add_handler(CommandHandler("generate", generate))
    
    updater.start_polling()
    updater.idle()
    
# def main():
#     print(sys.path)
#     soup = BeautifulSoup(get_poem(), 'html.parser')
#     author = soup.find('div', {'class': "entity-heading_subtitle"}).text
#     poem_title = soup.find('h1', {'class': "entity-heading_title"}).text
#     poem_year_element = soup.find('div', {'class': 'content-columns_footer'})
#     poem_year = f', {poem_year_element.text}' if poem_year_element is not None else ''
#     text = soup.find('div', {'class': 'content-columns_block'}).findAll('p')
#     paragraphs = ['\n'.join(str(x).replace('<p>', '').replace('</p>', '').split('<br/>')) for x in text]
#     poem_text = '\n\n'.join(paragraphs)
#     print(author)
#     print(poem_title)
#     print(poem_text)

#     message = f"""
# <b>{poem_title}</b>

# {poem_text}
    
# <i>{author}{poem_year}</i>"""

#     send_message('-1001512646284', message)


if __name__ == '__main__':
    main()

2021-11-03 19:52:22,276 - apscheduler.scheduler - INFO - Scheduler started


Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
Я не знаю слово "commands"
[57]
Я не знаю слово "commands"
[57]


2021-11-03 20:03:28,984 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/alexenia/opt/anaconda3/lib/python3.8/site-packages/telegram/ext/dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "/Users/alexenia/opt/anaconda3/lib/python3.8/site-packages/telegram/ext/handler.py", line 198, in handle_update
    return self.callback(update, context)
  File "<ipython-input-8-64df927782ca>", line 87, in generate_seq
    N = int(context.args[0])
IndexError: list index out of range
2021-11-03 20:03:32,266 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/alexenia/opt/anaconda3/lib/python3.8/site-packages/telegram/ext/dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "/Users/alexenia/opt/anaconda3/lib/python3.8/si

Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
Я не знаю слово ""
[57]
[57, 7471]
[57, 7471]
